# Compare Models

This notebook compares various GFW models based on the `measure_speed` and `measure_course` with each other
and with the models from Dalhousie University.  Note that the distance-to-shore cutoff was disabled in the
Dalhousie models, so none of the models compared here are using distance-to-shore as a feature.

In [1]:
from __future__ import print_function, division
%matplotlib inline
import sys
import numpy as np
sys.path.append('..')
import warnings; warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML, Markdown
import datetime
import pytz
from sklearn import metrics
import csv
import vessel_scoring.models
import vessel_scoring.evaluate_model
from vessel_scoring import utils
import vessel_scoring.data
from glob import glob
import os

sys.path.append("../../training-data-source")
import tools as trtools
sys.path.append("../../vessel-classification-pipeline/classification")
from  classification.utility import is_test

In [2]:
# Load the data. 
# ==============

# *First load the load the split datasets generated from PyBossa data*

paths = glob("../datasets/from_ranges/*.npz")
rngdata = {os.path.basename(p) : vessel_scoring.data.load_dataset_by_vessel(p) for p in paths
          if p.endswith('.measures.npz')}

# *Then load mappings MMSI to true / inferred gear type

def gear_name_of(x):
    return os.path.splitext(os.path.splitext(x)[0])[0][9:]

# Create a mapping of MMSI to true label
true_label_map = {}
for gear_type in rngdata:
    gear_name = gear_name_of(gear_type)
    for x in rngdata[gear_type][0]:
        true_label_map[x['mmsi']] = gear_name
        
# Create mapping of MMSI to inferred label
inferred_label_map = {}
with open("../../vessel-classification-pipeline copy/classification/inferred_labels.csv") as f:
    for row in csv.DictReader(f):
        inferred_label_map[int(row['mmsi'])] = row['label'].strip().replace('/', '_').replace(' ', '_')
        
# *Create datasets for training and testing*

# If this is true, test using inferred classes,
# otherwise use true classes
test_using_inferred = True

all_data = np.concatenate([x[0] for x in rngdata.values()])

test_data = {}
train_data = {}
all_data = np.concatenate([x[0] for x in rngdata.values()])
all_data['classification'] = (all_data['classification'] > 0.5)

for gear_type in rngdata:
    gear_name = gear_name_of(gear_type)
    
    # use the true gear types for the training data
    true_mask = [(true_label_map.get(int(x['mmsi'])) == gear_name and not is_test(int(x['mmsi']))) 
                     for x in all_data]
    true_mask = np.array(true_mask)
    if true_mask.sum():
        train_data[gear_type] = all_data[true_mask]    

    if test_using_inferred:
        # Use the inferred gear types for the test data
        test_mask = np.array(
            [(inferred_label_map.get(int(x['mmsi'])) == gear_name and is_test(int(x['mmsi'])))
                         for x in all_data])
    else:
        # Use the inferred gear types for the test data
        test_mask = np.array(
            [(true_label_map.get(int(x['mmsi'])) == gear_name and is_test(int(x['mmsi'])))
                         for x in all_data])
    if test_mask.sum():
        test_data[gear_type] = all_data[test_mask]


Warning, insufficient items to sample, returning all
Warning, insufficient items to sample, returning all


In [3]:
def compare_fishing_localization(true_ranges, inferred_ranges):

    all_mmsi = sorted(set(x.mmsi for x in true_ranges))

    true_by_mmsi = {}
    pred_by_mmsi = {}

    for mmsi in all_mmsi:
        true = np.array([x for x in true_ranges if x.mmsi == mmsi])
        inferred = np.array([x for x in inferred_ranges if x.mmsi == mmsi])

        # Determine minutes from start to finish of this mmsi, create an array to
        # hold results and fill with -1 (unknown)
        _, start, end, _ = true[0]
        for (_, s, e, _) in true[1:]:
            start = min(start, s)
            end = max(end, e)
        start_min = datetime_to_minute(start)
        end_min = datetime_to_minute(end)
        
        minutes = np.empty([end_min - start_min + 1, 2], dtype=int)
        minutes.fill(-1)

        # Fill in minutes[:, 0] with known true / false values
        for (_, s, e, is_fishing) in true:
            s_min = datetime_to_minute(s)
            e_min = datetime_to_minute(e)
            for m in range(s_min - start_min, e_min - start_min + 1):
                minutes[m, 0] = is_fishing

        # fill in minutes[:, 1] with in inferred values
        for (_, s, e, is_fishing) in inferred:
            s_min = datetime_to_minute(s)
            e_min = datetime_to_minute(e)
            for m in range(s_min - start_min, e_min - start_min + 1):
                if 0 <= m < len(minutes):
                    minutes[m, 1] = is_fishing       
 
        mask = ((minutes[:, 0] != -1) & (minutes[:, 1] != -1))

        if mask.sum():
            true_by_mmsi[mmsi] = minutes[mask, 0]
            pred_by_mmsi[mmsi] = minutes[mask, 1]
            
    return true_by_mmsi, pred_by_mmsi

def ranges_from_ais(ais, is_fishing):
    points = [trtools.Point(x['mmsi'], 
                            datetime.datetime.utcfromtimestamp(x['timestamp']).replace(tzinfo=pytz.utc), is_fishing[i])
              for i, x in enumerate(ais)]
    return trtools.ranges_from_points(points)

def datetime_to_minute(dt):
    timestamp = (dt - datetime.datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds()
    return int(timestamp // 60)

def model_true_inferred(mdl, test_data):
    check =  mdl.predict_proba(test_data)[:,1]
    
    inferred_ranges = list(ranges_from_ais(test_data, mdl.predict_proba(test_data)[:,1] > 0.5))
    true_ranges = list(ranges_from_ais(test_data, test_data['classification'] > 0.5))
    
    true, inferred = compare_fishing_localization(true_ranges, inferred_ranges)
    
    y_true = np.concatenate(true.values())
    y_pred = np.concatenate(inferred.values())
    
    return y_true, y_pred

def model_metrics(y_true, y_pred):
    
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    
    return precision, recall, accuracy, f1, y_true, y_pred
    

In [4]:
models = [
        ('Random Forest with Distances', vessel_scoring.random_forest_model.RandomForestModel(
                colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
                                measures=['speed', 'distance_from_shore', 'distance_from_port']))),
        ('Logistic', vessel_scoring.logistic_model.LogisticModel(order=6,
                colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
                                measures=['measure_speed']))),
] 

for name, mdl in models:

    lines = ["|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|",
             "|-----|-----|------|---------|------|--------|--------|"]

    all_true = []
    all_pred = []
    all_mmsi = []

    for gear in sorted(rngdata):
        title = gear.split('.')[0].split('_',1)[1].replace('_', ' ')

        gear_name = os.path.splitext(os.path.splitext(gear)[0])[0][9:]

        print(".", end="")
        
        if gear not in test_data:
            continue
        
        trained = vessel_scoring.models.train_model_on_data(mdl, train_data[gear])

        y_true, y_pred =  model_true_inferred(trained, test_data[gear])

        all_mmsi.append(test_data[gear]['mmsi'].astype(int))
        all_true.append(y_true)
        all_pred.append(y_pred)

        
    # Now remap the data from the inferred labels to the true labels so that
    # we can place them in the correct buckets

    mmsi_all = np.concatenate(all_mmsi)
    y_true_all = np.concatenate(all_true)
    y_pred_all = np.concatenate(all_pred)
    
    for gear_type in sorted(set(true_label_map.values())):
        # Create a mask of all points with MMSI corresponding to this TRUE label
        mask = [(true_label_map.get(x) == gear_type and is_test(x)) for x in mmsi_all]
        mask = np.array(mask)
        # Fine y_true, y_pred based using the mask and compute metrics
        y_true = y_true_all[mask]
        y_pred = y_pred_all[mask]
        mmsi_count = len(set(mmsi_all[mask]))
        n_points = len(mmsi_all[mask])
        accuracy = metrics.accuracy_score(y_true, y_pred)
        precision = metrics.precision_score(y_true, y_pred)
        recall = metrics.recall_score(y_true, y_pred)
        f1 = metrics.f1_score(y_true, y_pred)
        # Add to Markdown table
        lines.append('|{}|{}|{}|'.format(gear_type, mmsi_count, n_points) + '|'.join(['{:.2f}'.format(x) for x in
                                    [precision, recall, accuracy, f1]]) + '|')

    # Compute metrics over all test points.
    mask = np.array([(is_test(x)) for x in mmsi_all])
    y_true = y_true_all[mask]
    y_pred = y_pred_all[mask]

    mmsi_count =len(set(mmsi_all[mask]))
    n_points = len(mmsi_all[mask])
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)

    lines.append("|     |     |     |     |     |")

    lines.append('|Overall|{}|{}|'.format(mmsi_count, n_points) + '|'.join(['{:.2f}'.format(x) for x in
                                [precision, recall, accuracy, f1]]) + '|')

    display(HTML("<h2>{}</h2>".format(name)))

    display(Markdown('\n'.join(lines)))

.......

|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|
|-----|-----|------|---------|------|--------|--------|
|Longliners|40|122240|0.91|0.87|0.90|0.89|
|Pole_and_Line|4|8197|0.82|0.93|0.82|0.87|
|Pots_and_Traps|5|10082|0.73|0.99|0.88|0.84|
|Purse_seines|15|17595|0.76|1.00|0.84|0.86|
|Set_gillnets|8|11448|0.80|0.98|0.82|0.88|
|Trawlers|15|33446|0.98|0.94|0.96|0.96|
|Trollers|1|2768|1.00|0.72|0.72|0.84|
|     |     |     |     |     |
|Overall|88|205776|0.88|0.91|0.89|0.89|

.......

|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|
|-----|-----|------|---------|------|--------|--------|
|Longliners|40|122240|0.89|0.93|0.92|0.91|
|Pole_and_Line|4|8197|0.71|0.90|0.69|0.79|
|Pots_and_Traps|5|10082|0.68|0.97|0.85|0.80|
|Purse_seines|15|17595|0.76|1.00|0.84|0.86|
|Set_gillnets|8|11448|0.77|0.99|0.79|0.87|
|Trawlers|15|33446|0.92|0.94|0.92|0.93|
|Trollers|1|2768|0.90|0.87|0.81|0.89|
|     |     |     |     |     |
|Overall|88|205776|0.85|0.94|0.89|0.90|